<center><a href="https://www.featuretools.com/"><img src="img/featuretools-logo.png" width="400" height="200" /></a></center>

<h2> An Advanced Featuretools Approach with Custom Primitives</h2>
<p>The following tutorial illustrates an advanced featuretools model for the NYC Taxi Trip Duration competition on Kaggle using our custom primitive API. You will need to download the following five files into a `data` folder in this repository.</p>

<h2>Step 1: Download raw data </h2>
+ <a href="https://www.kaggle.com/c/nyc-taxi-trip-duration/data">test.csv and train.csv</a>
+ <a href="https://www.kaggle.com/oscarleo/new-york-city-taxi-with-osrm/data">fastest_routes train1, train2 and test</a>

The functions used here don't appear until Featuretools version 0.1.9. If you're using a lower version you will get an error.

In [1]:
import featuretools as ft
import pandas as pd
import numpy as np
import utils
ft.__version__

'0.2.1'

<h2>Step 2: Prepare the Data </h2>

In [2]:
TRAIN_DIR = "data/train.csv"
TEST_DIR = "data/test.csv"

data_train, data_test = utils.read_data(TRAIN_DIR, TEST_DIR, nrows=None)

# Load other data https://www.kaggle.com/oscarleo/new-york-city-taxi-with-osrm
fastest_routes_part1 = pd.read_csv('data/fastest_routes_train_part_1.csv', 
                                   usecols=['id', 'number_of_steps'])
fastest_routes_part2 = pd.read_csv('data/fastest_routes_train_part_2.csv', 
                                   usecols=['id', 'number_of_steps'])
fastest_routes_train = pd.concat((fastest_routes_part1, fastest_routes_part2))

fatest_routes_test = pd.read_csv('data/fastest_routes_test.csv',
                                 usecols=['id', 'number_of_steps'])

The `fastest_routes` dataset has information on the shortest path between two coordinates in NYC. We can merge it in with our dataset here and then merge together our test and train datasets after marking them as such.

In [3]:
data_train = data_train.merge(fastest_routes_train, how='left', on='id')
data_test = data_test.merge(fatest_routes_test, how='left', on='id')

# Make a train/test column
data_train['test_data'] = False
data_test['test_data'] = True

# Combine the data and convert some strings
data = pd.concat([data_train, data_test])
data.head(5)

,dropoff_latitude,dropoff_longitude,id,number_of_steps,passenger_count,pickup_datetime,pickup_latitude,pickup_longitude,store_and_fwd_flag,test_data,trip_duration,vendor_id
0,40.765602,-73.964630,id2875421,5.0,1,2016-03-14 17:24:55,40.767937,-73.982155,False,False,455.0,2
1,40.731152,-73.999481,id2377394,6.0,1,2016-06-12 00:43:35,40.738564,-73.980415,False,False,663.0,1
2,40.710087,-74.005333,id3858529,16.0,1,2016-01-19 11:35:24,40.763939,-73.979027,False,False,2124.0,2
3,40.706718,-74.012268,id3504673,4.0,1,2016-04-06 19:32:31,40.719971,-74.010040,False,False,429.0,2
4,40.782520,-73.972923,id2181028,5.0,1,2016-03-26 13:30:55,40.793209,-73.973053,False,False,435.0,2


## Step 3: Custom Primitives
The custom primitive API is new to Featuretools version 0.1.9. Our workflow will be as follows:
1. Make a new LatLong class which is a tuple of a latitude and longitude
2. Define some functions for LatLong
3. Make new primitives from those functions

For the first step, we'll make new columns for the pickup and dropoff locations which are tuples.

In [4]:
data["pickup_latlong"] = data[['pickup_latitude', 'pickup_longitude']].apply(tuple, axis=1)
data["dropoff_latlong"] = data[['dropoff_latitude', 'dropoff_longitude']].apply(tuple, axis=1)
data = data.drop(["pickup_latitude", "pickup_longitude", "dropoff_latitude", "dropoff_longitude"], axis = 1)

<p>We can define the `pickup_latlong` to be a `TripStart` type which will be built on our `LatLong` type. This is a way to tell DFS that certain primitives are only important in one direction, from the beginning of a trip to the end. 

Marking separate types for `TripStart` and `TripEnd` can be skipped as of Featuretools version 0.1.17 with the `commutative` functionality in DFS.</p>

In [5]:
from featuretools import variable_types as vtypes

class LatLong(vtypes.Variable):
    _dtype_repr = "latlong"

class TripStart(LatLong):
    _dtype_repr = "latlong"

class TripEnd(LatLong):
    _dtype_repr = "latlong"
    
trip_variable_types = {
    'passenger_count': vtypes.Ordinal, 
    'vendor_id': vtypes.Categorical,
    'pickup_latlong': TripStart,
    'dropoff_latlong': TripEnd,
}

es = ft.EntitySet("taxi")

es.entity_from_dataframe(entity_id="trips",
                         dataframe=data,
                         index="id",
                         time_index='pickup_datetime',
                         variable_types=trip_variable_types)

es.normalize_entity(base_entity_id="trips",
                    new_entity_id="vendors",
                    index="vendor_id")

es.normalize_entity(base_entity_id="trips",
                    new_entity_id="passenger_cnt",
                    index="passenger_count")

cutoff_time = es['trips'].df[['id', 'pickup_datetime']]

<p>Next, we can create primitives for our new `pickup_latlong` and `dropoff_latlong` which are of type `TripStart` and `TripEnd`. 

The distance between two `LatLong` types is most accurately represented by the Haversine distance: the nearest length along a sphere. We can also define the Cityblock distance (also called the taxicab metric), which takes into account that we can't move diagonally through buildings in Manhattan.</p>

In [6]:
from featuretools.primitives import make_agg_primitive, make_trans_primitive

def haversine(latlong1, latlong2):
    lat_1s = np.array([x[0] for x in latlong1])
    lon_1s = np.array([x[1] for x in latlong1])
    lat_2s = np.array([x[0] for x in latlong2])
    lon_2s = np.array([x[1] for x in latlong2])
    lon1, lat1, lon2, lat2 = map(np.radians, [lon_1s, lat_1s, lon_2s, lat_2s])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2.0)**2
    km = 6367 * 2 * np.arcsin(np.sqrt(a))
    return km

def cityblock(latlong1, latlong2):
    lon_dis = haversine(latlong1, latlong2)
    lat_dist = haversine(latlong1, latlong2)
    return lon_dis + lat_dist

def bearing(latlong1, latlong2):
    lat1 = np.array([x[0] for x in latlong1])
    lon1 = np.array([x[1] for x in latlong1])
    lat2 = np.array([x[0] for x in latlong2])
    lon2 = np.array([x[1] for x in latlong2])
    delta_lon = np.radians(lon2 - lon1)
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    x = np.cos(lat2) * np.sin(delta_lon)
    y = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(delta_lon)
    return np.degrees(np.arctan2(x, y))

We can also make som primitives directly from `LatLong` types. In particular, we'll almost certainly want to extract the Latitude, the Longitude and the central point between two such coordinates.

In [7]:
def latitude(latlong):
    return np.array([x[0] for x in latlong])

def longitude(latlong):
    return np.array([x[1] for x in latlong])

def center(latlong1, latlong2):
    return np.array([[.5*(latlong1[i][0] + latlong2[i][0]),
                          .5*(latlong1[i][1] + latlong2[i][1])]
                     for i, x in enumerate(latlong1)])

<p>Now lets make primitives from those functions!</p>

In [8]:
Bearing = make_trans_primitive(function=bearing,
                               input_types=[TripStart, TripEnd],
                               return_type=vtypes.Numeric)

Cityblock = make_trans_primitive(function=cityblock,
                                 input_types=[TripStart, TripEnd],
                                 return_type=vtypes.Numeric)

Haversine = make_trans_primitive(function=haversine,
                                 input_types=[TripStart, TripEnd],
                                 return_type=vtypes.Numeric)

Latitude = make_trans_primitive(function=latitude, 
                                input_types=[LatLong],
                                return_type=vtypes.Numeric)

Longitude = make_trans_primitive(function=longitude,
                                 input_types=[LatLong],
                                 return_type=vtypes.Numeric)

Center = make_trans_primitive(function=center,
                              input_types=[TripStart, TripEnd],
                              return_type=LatLong)

## Seed Features
A "Seed Feature" is a feature that we don't necessarily plan on using as a primitive outside of this problem. Deep Feature Synthesis allows for automatic stacking on top of seed features as well. 

Here we can define some seed features which are relevant directly to predicting trip duration. The feature `Geobox` finds if a latlong is in a rectangle defined by two coordinate pairs. The feature `Numbox` is given here as a similar example, though it's more straightforward to define each of those seed features directly.

In [9]:
def geobox(latlong, bottomleft=None, topright=None):
    lat = np.array([x[0] for x in latlong])
    lon = np.array([x[1] for x in latlong])
    boxlats = [bottomleft[0],topright[0]]
    boxlongs = [bottomleft[1], topright[1]]
    output = []
    for i, name in enumerate(lat):
        if (min(boxlats) <= lat[i] and lat[i] <= max(boxlats) and 
                min(boxlongs) <= lon[i] and lon[i] <= max(boxlongs)):
            output.append(True)
        else: 
            output.append(False)
    return output

def numbox(number, less=None, more=None):
    output = []
    for i, name in enumerate(number):
        if less<=number[i] and number[i]<=more:
            output.append(True)
        else: 
            output.append(False)
    return output

def geobox_get_name(self):
    return u"GEOBOX({}, {}, {})".format(self.base_features[0].get_name(),
                                        str(self.kwargs['bottomleft']),
                                        str(self.kwargs['topright']))

Geobox = make_trans_primitive(function=geobox,
                              input_types=[LatLong],
                              return_type=vtypes.Boolean,
                              cls_attributes={"get_name": geobox_get_name})

def numbox_get_name(self):
    return u"NUMBOX({}, {}, {})".format(self.base_features[0].get_name(),
                                        str(self.kwargs['less']), 
                                        str(self.kwargs['more']))

Numbox = make_trans_primitive(function=numbox,
                              input_types=[vtypes.Ordinal],
                              return_type = vtypes.Boolean,
                              cls_attributes = {"get_name": numbox_get_name})

With those functions defined, we can now implement a geographic box around JFK airport with the `GEOBOX` primitive. We can also group together some times of day with `NUMBOX`.

In [10]:
from featuretools.primitives import (Feature, Hour)

seed_features = []

jfk_pick = Geobox(es['trips']['pickup_latlong'], 
                  bottomleft = (40.62, -73.85), 
                  topright = (40.70, -73.75))
jfk_drop = Geobox(es['trips']['dropoff_latlong'],
                  bottomleft = (40.62, -73.85), 
                  topright = (40.70, -73.75))

yonkers_pick = Geobox(es['trips']['pickup_latlong'], 
                      bottomleft = (40.70, -73.97), 
                      topright = (40.77, -73.9))
yonkers_drop = Geobox(es['trips']['dropoff_latlong'],
                      bottomleft = (40.70, -73.97), 
                      topright = (40.77, -73.9))

     
rush = Numbox(Hour(es["trips"]["pickup_datetime"]), less = 7, more = 11)
noon = Numbox(Hour(es["trips"]["pickup_datetime"]), less = 11, more = 13)
night = Numbox(Hour(es["trips"]["pickup_datetime"]), less = 18, more = 23)

seed_features = [jfk_pick, jfk_drop,
                 yonkers_pick, yonkers_drop, rush, noon, night]

## Step 4: Using custom primitives in DFS

Let's see what features are created if we run DFS now:

In [11]:
agg_primitives = []
trans_primitives = [Bearing, Haversine, Cityblock, Center, Latitude, Longitude]

# calculate feature_matrix using deep feature synthesis
features = ft.dfs(entityset=es,
                  target_entity="trips",
                  trans_primitives=trans_primitives,
                  agg_primitives=agg_primitives,
                  drop_contains=['trips.test_data'],
                  verbose=True,
                  cutoff_time=cutoff_time,
                  approximate='36d',
                  seed_features=seed_features,
                  max_depth=3,
                  max_features=40,
                  features_only=True)
features

Building features: 31it [00:00, 11091.31it/s]


[<Feature: passenger_count>,
 <Feature: number_of_steps>,
 <Feature: store_and_fwd_flag>,
 <Feature: vendor_id>,
 <Feature: test_data>,
 <Feature: trip_duration>,
 <Feature: LATITUDE(dropoff_latlong)>,
 <Feature: LONGITUDE(pickup_latlong)>,
 <Feature: BEARING(pickup_latlong, dropoff_latlong)>,
 <Feature: GEOBOX(dropoff_latlong, (40.7, -73.97), (40.77, -73.9))>,
 <Feature: GEOBOX(pickup_latlong, (40.7, -73.97), (40.77, -73.9))>,
 <Feature: HAVERSINE(pickup_latlong, dropoff_latlong)>,
 <Feature: GEOBOX(pickup_latlong, (40.62, -73.85), (40.7, -73.75))>,
 <Feature: CITYBLOCK(pickup_latlong, dropoff_latlong)>,
 <Feature: LONGITUDE(dropoff_latlong)>,
 <Feature: GEOBOX(dropoff_latlong, (40.62, -73.85), (40.7, -73.75))>,
 <Feature: LATITUDE(pickup_latlong)>,
 <Feature: NUMBOX(HOUR(pickup_datetime), 18, 23)>,
 <Feature: LONGITUDE(CENTER(pickup_latlong, dropoff_latlong))>,
 <Feature: LATITUDE(CENTER(pickup_latlong, dropoff_latlong))>,
 <Feature: NUMBOX(HOUR(pickup_datetime), 11, 13)>,
 <Feature:

Our new features were applied exactly where they should be! The distances were only calcuated between the pickup_latlong and the dropoff_latlong, while primitives like `LATITUDE` were calculated on both latlong columns.

For this dataset, we have approximately 2 million distinct cutoff times, times at which DFS will have to recalculate aggregation primitives. Calculating those features at specific times every hour and using that number instead will save computation time and perhaps not lose too much information. The `approximate` parameter in DFS lets you specify a window size to use when approximating.

With the features from before, we calculate the whole feature matrix:

In [12]:
agg_primitives = ['Sum', 'Mean', 'Median', 'Std', 'Count', 'Min', 'Max', 'Num_Unique', 'Skew']
trans_primitives = [Bearing, Haversine, Cityblock, Center, Latitude, Longitude, 
                    Day, Hour, Minute, Month, Weekday, Week, Weekend]

# this allows us to create features that are conditioned on a second value before we calculate.
es.add_interesting_values()

# calculate feature_matrix using deep feature synthesis
feature_matrix, features = ft.dfs(entityset=es,
                                  target_entity="trips",
                                  trans_primitives=trans_primitives,
                                  agg_primitives=agg_primitives,
                                  drop_contains=['trips.test_data'],
                                  verbose=True,
                                  cutoff_time=cutoff_time,
                                  approximate='36d',
                                  seed_features=seed_features,
                                  max_depth=4)
feature_matrix.head()

Building features: 404it [00:00, 4283.37it/s]
Progress: 100%|██████████| 6/6 [15:18<00:00, 153.12s/cutoff time]


,store_and_fwd_flag,number_of_steps,test_data,trip_duration,vendor_id,passenger_count,LONGITUDE(pickup_latlong),MONTH(pickup_datetime),"HAVERSINE(pickup_latlong, dropoff_latlong)","CITYBLOCK(pickup_latlong, dropoff_latlong)",...,"vendors.MEAN(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","vendors.MIN(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","vendors.SKEW(trips.LATITUDE(CENTER(pickup_latlong, dropoff_latlong)))","vendors.MAX(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","vendors.MEAN(trips.LATITUDE(CENTER(pickup_latlong, dropoff_latlong)))","passenger_cnt.MAX(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","vendors.STD(trips.LATITUDE(CENTER(pickup_latlong, dropoff_latlong)))","passenger_cnt.MEDIAN(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","passenger_cnt.SUM(trips.LONGITUDE(CENTER(pickup_latlong, dropoff_latlong)))","passenger_cnt.MIN(trips.LATITUDE(CENTER(pickup_latlong, dropoff_latlong)))"
id,,,,,,,,,,,,,,,,,,,,,
id0000002,False,14.0,True,NaN,2,1,-73.983673,1,6.817899,13.635799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id0000037,False,5.0,False,466.0,2,2,-73.986992,1,2.091157,4.182313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id0000107,False,20.0,False,1215.0,2,1,-73.977898,1,8.920172,17.840345,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id0000122,False,7.0,False,301.0,2,4,-73.976410,1,1.029770,2.059540,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
id0000157,False,5.0,False,182.0,2,2,-73.973923,1,0.925471,1.850943,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 5: Build the Model

<p>As before, we need to retrieve our labels for the train dataset, so we should merge our current feature matrix with the original dataset. </p>

<p>We use the `log` of the trip duration since that measure is better at distinguishing distances within the city</p>

In [13]:
# separates the whole feature matrix into train data feature matrix, train data labels, and test data feature matrix 
X_train, labels, X_test = utils.get_train_test_fm(feature_matrix)
labels = np.log(labels.values + 1)

In [14]:
model = utils.train_xgb(X_train, labels)

[0]	train-rmse:4.99586	valid-rmse:4.99578
Multiple eval metrics have been passed: 'valid-rmse' will be used for early stopping.

Will train until valid-rmse hasn't improved in 60 rounds.
[10]	train-rmse:0.909667	valid-rmse:0.911437
[20]	train-rmse:0.393896	valid-rmse:0.398602
[30]	train-rmse:0.35112	valid-rmse:0.357798
[40]	train-rmse:0.338103	valid-rmse:0.346866
[50]	train-rmse:0.330987	valid-rmse:0.341707
[60]	train-rmse:0.325593	valid-rmse:0.337964
[70]	train-rmse:0.321898	valid-rmse:0.335724
[80]	train-rmse:0.318819	valid-rmse:0.33381
[90]	train-rmse:0.316674	valid-rmse:0.332933
[100]	train-rmse:0.313719	valid-rmse:0.331824
[110]	train-rmse:0.311709	valid-rmse:0.331004
[120]	train-rmse:0.309871	valid-rmse:0.330386
[130]	train-rmse:0.308356	valid-rmse:0.329967
[140]	train-rmse:0.307061	valid-rmse:0.32964
[150]	train-rmse:0.305756	valid-rmse:0.329084
[160]	train-rmse:0.304798	valid-rmse:0.328806
[170]	train-rmse:0.304052	valid-rmse:0.328622
[180]	train-rmse:0.303319	valid-rmse:0.3284

In [15]:
submission = utils.predict_xgb(model, X_test)
submission.head(5)

,trip_duration
id,
id0000002,1078.245972
id0000199,2036.927246
id0000446,841.454407
id0000587,1364.801514
id0000604,237.510635


In [16]:
submission.to_csv('trip_duration_ft_custom_primitives.csv', index=True, index_label='id')

<dl>
<dt>This solution:</dt>
<dd>&nbsp; &nbsp; Received a score of 0.38694 on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Placed 250 out of 1257.</dd>
<dd>&nbsp; &nbsp; Beat 80% of competitors on the Kaggle competition.</dd>
<dd>&nbsp; &nbsp; Scored 39% better than the baseline solution</dd>
<dd>&nbsp; &nbsp; Had a modeling RMSLE of 0.32749</dd>
</dl>

December 27, 2017.

<h2>Additional Analysis</h2>
<p>Lets look at how important the features we created were for the model.</p>

In [17]:
feature_names = X_train.columns.values
ft_importances = utils.feature_importances(model, feature_names)
ft_importances[:50]

,feature_name,importance
134,"HAVERSINE(pickup_latlong, dropoff_latlong)",3714.0
132,LONGITUDE(pickup_latlong),3654.0
13,"LATITUDE(CENTER(pickup_latlong, dropoff_latlong))",3612.0
122,LATITUDE(pickup_latlong),3527.0
127,"BEARING(pickup_latlong, dropoff_latlong)",3525.0
119,LATITUDE(dropoff_latlong),3422.0
34,"LONGITUDE(CENTER(pickup_latlong, dropoff_latlo...",3376.0
137,LONGITUDE(dropoff_latlong),3259.0
123,HOUR(pickup_datetime),2433.0
135,"CITYBLOCK(pickup_latlong, dropoff_latlong)",2012.0
